# Regresión de Deming
-------------------------------

Este ejercicio muestra como usar TensorFlow para resolver la regresión lineal de Deming.

$y = Ax + b$

Usaremos los datos de Iris, específicamente:

y = Sepal Length y x = Petal Width.

La regresión de Deming también se denomina mínimos cuadrados totales, en los cuales minimizamos la distancia más corta desde la línea predicha y los puntos reales (x, y).

Si la regresión lineal de mínimos cuadrados minimiza la distancia vertical a la línea, la regresión de Deming minimiza la distancia total a la línea. Este tipo de regresión minimiza el error en los valores y y los valores x. Vea la figura de abajo para una comparación.

<img src="images/05_demming_vs_linear_reg.png" width="512">

Para implementar esto en TensorFlow, comenzamos cargando las bibliotecas necesarias.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn import datasets
from tensorflow.python.framework import ops
ops.reset_default_graph()

Iniciar una sesión computacional del grafo:

In [ ]:
sess = tf.Session()

# Establecer una semilla al azar
tf.set_random_seed(42)
np.random.seed(42)

Cargamos los datos de Iris:

In [ ]:
# Cargar los datos
# iris.data = [(Sepal Length, Sepal Width, Petal Length, Petal Width)]
iris = datasets.load_iris()
x_vals = np.array([x[3] for x in iris.data]) # Petal Width
y_vals = np.array([y[0] for y in iris.data]) # Sepal Length

A continuación, declaramos el tamaño del lote, los marcadores de posición del modelo, las variables del modelo y las operaciones del modelo.

In [ ]:
# Declarar el tamaño del lote
batch_size = 125

# Inicializar los marcadores de posición
x_data = tf.placeholder(shape=[None, 1], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# Crear las variables para la Regresión lineal
A = tf.Variable(tf.random_normal(shape=[1,1]))
b = tf.Variable(tf.random_normal(shape=[1,1]))

# Declarar las operaciones del modelo
model_output = tf.add(tf.matmul(x_data, A), b)

Para la pérdida demming, queremos calcular:

$$ \frac{\left| A \cdot x + b - y \right|}{\sqrt{A^{2} + 1}} $$

Lo que nos dará la distancia más corta entre un punto (x, y) y la línea predicha, $A \cdot x + b$.

In [ ]:
# Declarar la función de pérdida para Deming
deming_numerator = tf.abs(tf.subtract(tf.add(tf.matmul(x_data, A), b), y_target))
deming_denominator = tf.sqrt(tf.add(tf.square(A),1))
loss = tf.reduce_mean(tf.truediv(deming_numerator, deming_denominator))

A continuación declaramos la función de optimización e inicializamos todas las variables del modelo.

In [ ]:
# Declarar el optimizador
my_opt = tf.train.GradientDescentOptimizer(0.25)
train_step = my_opt.minimize(loss)

# Inicializar las variables
init = tf.global_variables_initializer()
sess.run(init)

Ahora entrenamos nuestra regresión de Deming para 250 iteraciones.

In [ ]:
# Ciclo de entrenamiento
loss_vec = []
for i in range(1500):
    rand_index = np.random.choice(len(x_vals), size=batch_size)
    rand_x = np.transpose([x_vals[rand_index]])
    rand_y = np.transpose([y_vals[rand_index]])
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
    temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
    loss_vec.append(temp_loss)
    if (i+1)%100==0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)) + ' b = ' + str(sess.run(b)))
        print('Loss = ' + str(temp_loss))

Recuperar los coeficientes óptimos (pendiente e intersección).

In [ ]:
# Consigue los coeficientes óptimos.
[slope] = sess.run(A)
[y_intercept] = sess.run(b)

# Obtener la mejor línea de ajuste
best_fit = []
for i in x_vals:
  best_fit.append(slope*i+y_intercept)

Here is matplotlib code to plot the best fit Deming regression line and the Demming Loss.

In [ ]:
# Graficar el resultado
plt.plot(x_vals, y_vals, 'o', label='Data Points')
plt.plot(x_vals, best_fit, 'r-', label='Best fit line', linewidth=3)
plt.legend(loc='upper left')
plt.title('Sepal Length vs Petal Width')
plt.xlabel('Petal Width')
plt.ylabel('Sepal Length')
plt.show()

# Graficar la pérdida con el tiempo
plt.plot(loss_vec, 'k-')
plt.title('Deming Loss per Generation')
plt.xlabel('Iteration')
plt.ylabel('Deming Loss')
plt.show()